In [1]:
import os
import pycolmap
import sys
import time
import pickle

# from tabilize import *

# import evo.core.lie_algebra as lie
# from evo.core import trajectory
# from evo.tools import plot, file_interface, log
# from evo.core import lie_algebra, sync, metrics

import matplotlib.pyplot as plt
import numpy as np
# import plotly.graph_objects as go
# from plotly.offline import plot

import networkx as nx
import matplotlib.pyplot as plt
# from skimage import graph
# import sklearn.cluster

# from sklearn import metrics
# from tqdm import tqdm

from IPython.display import clear_output
from helpers import *
import locale


In [2]:
def rankify(x, order):
    # Turn a vector of values into a list of ranks, while handling ties.
    assert len(x.shape) == 1
    if order == 0:
        return np.full_like(x, 1e5, dtype=np.int32)
    u = np.sort(np.unique(x))
    if order == 1:
        u = u[::-1]
    r = np.zeros_like(x, dtype=np.int32)
    for ui, uu in enumerate(u):
        mask = x == uu
        r[mask] = ui
    return np.int32(r)

In [3]:
# Set up the available datasets here
datasets_full = {
    "KITTI":
        # From IMU. The error for the 00 is too large and 03 is missing
        ["01", "02", "04", "05", "06", "07", "08", "09", "10"],
        # ["01", "02", "04", "05", ],
        # ["01","04"],
    "lamar": [
            "2022-01-18-12-41-23-216", 
            "2022-06-20-14-33-15-200",
            "2022-08-30-12-11-39-585",
            "2022-08-30-14-37-23-164", 
        ],
    "1dsfm_colmap": [
            'Alamo',
            # 'Gendarmenmarkt',
            'Madrid_Metropolis',
            'Roman_Forum',
            'Tower_of_London',
        ],
    "euroc" : [
            "MH_01_easy",
            "MH_02_easy",
            "MH_03_medium",
            "MH_04_difficult",
            "MH_05_difficult",
            "V1_01_easy",
            "V1_02_medium",
            "V1_03_difficult",
            "V2_01_easy",
            "V2_02_medium",
            "V2_03_difficult"
    ],
    "lamar_map" : [
            # "CAB",
            # "HGE",
            "LIN"
    ],
    "eth3d_slam": [
        "cables_1",
        "cables_2",
        "cables_3",
        "camera_shake_1",
        "camera_shake_2",
        "camera_shake_3",
        "ceiling_1",
        "ceiling_2",
        "desk_3",
        "desk_changing_1",
        "einstein_1",
        "einstein_2",
        "einstein_dark",
        "einstein_flashlight",
        "einstein_global_light_changes_1",
        "einstein_global_light_changes_2",
        "einstein_global_light_changes_3",
        "kidnap_1",
        # "kidnap_dark",
        "large_loop_1",
        "mannequin_1",
        "mannequin_3",
        "mannequin_4",
        "mannequin_5",
        "mannequin_7",
        "mannequin_face_1",
        "mannequin_face_2",
        "mannequin_face_3",
        "mannequin_head",
        "motion_1",
        "planar_2",
        "planar_3",
        "plant_1",
        "plant_2",
        "plant_3",
        "plant_4",
        "plant_5",
        # "plant_dark",
        "plant_scene_1",
        "plant_scene_2",
        "plant_scene_3",
        "reflective_1",
        "repetitive",
        "sfm_bench",
        "sfm_garden",
        "sfm_house_loop",
        "sfm_lab_room_1",
        "sfm_lab_room_2",
        "sofa_1",
        "sofa_2",
        "sofa_3",
        "sofa_4",
        # "sofa_dark_1",
        # "sofa_dark_2",
        # "sofa_dark_3",
        # "sofa_shake",
        "table_3",
        "table_4",
        "table_7",
        "vicon_light_1",
        "vicon_light_2"
    ],

    "eth3d_dslr": [
        "courtyard",
        "delivery_area",
        "electro",
        "facade",
        "kicker",
        "meadow",
        "office",
        "pipes",
        "playground",
        "relief",
        "relief_2",
        "terrace",
        "terrains"
    ],
    "eth3d_rig" : [
        "delivery_area",
        "electro",
        "forest",
        "playground",
        "terrains"
    ],
    'mip360': [
        "bicycle",
        "bonsai",
        "counter",
        "garden",
        "kitchen",
        "room",
        "stump"
    ],
    'eth3d_dslr_test': [
        "botanical_garden",
        "boulders",
        "bridge",
        "door",
        "exhibition_hall",
        "lecture_room",
        "living_room",
        "lounge",
        "observatory",
        "old_computer",
        "statue",
        "terrace_2",
    ],
    'imc_haiper': [
        "bike",
        "chairs",
        "fountain"
    ],
    'imc_heritage': [
        "cyprus",
        "dioscuri",
        "wall"
    ],
    'imc_urban': [
        "kyiv-puppet-theater"
    ],
    'imc_phototourism': [
        'brandenburg_gate',
        'british_museum',
        'buckingham_palace',
        'colosseum_exterior',
        'grand_place_brussels',
        'lincoln_memorial_statue',
        'notre_dame_front_facade',
        'pantheon_exterior',
        'piazza_san_marco',
        'sacre_coeur',
        'sagrada_familia',
        'st_pauls_cathedral',
        'st_peters_square',
        'taj_mahal',
        # 'temple_nara_japan',
        'trevi_fountain'
    ],
    '1dsfm_ori': [
        # "ArtsQuad",
        # "Union_Square",
        "Vienna_Cathedral"
    ],
    'flowmap': [
        "co3d_bench",
        "co3d_hydrant",
        "llff_fern",
        "llff_flower",
        "llff_fortress",
        "llff_horns",
        # "llff_leaves",
        "llff_orchids",
        "llff_room",
        "llff_trex",
        "mipnerf360_bonsai",
        "mipnerf360_counter",
        # "mipnerf360_garden",
        "mipnerf360_kitchen",
        # "tandt_auditorium",
        # "tandt_ballroom",
        "tandt_barn",
        "tandt_caterpillar",
        "tandt_church",
        "tandt_courthouse",
        # "tandt_courtroom",
        "tandt_family",
        "tandt_francis",
        "tandt_horse",
        "tandt_ignatius",
        # "tandt_lighthouse",
        "tandt_m60",
        # "tandt_meetingroom",
        "tandt_museum",
        # "tandt_palace",
        "tandt_panther",
        "tandt_playground",
        # "tandt_temple",
        "tandt_train",
        "tandt_truck"
    ],
    "strecha": [
        "Castle",
        "Castle_large",
        "Entry",
        "Fountain",
        "Herzjesu",
        "Herzjesu_large",
    ],
    "imc2024": [
        # "church",
        "dioscuri",
        # "lizard",
        "multi-temporal-temple-baalshamin",
        # "pond",
        # "transp_obj_glass_cup",
        # "transp_obj_glass_cylinder"
    ],
    "imc2021_test": [
        "british_museum",
        "florence_cathedral_side",
        "lincoln_memorial_statue",
        "london_bridge",
        "milan_cathedral",
        "mount_rushmore",
        "piazza_san_marco",
        "sagrada_familia",
        "st_pauls_cathedral"
    ],
    "imc2021_val": [
        "reichstag",
        "sacre_coeur",
        "st_peters_square"
    ],
    "eth3d_raw_dslr": [
        "courtyard",
        # "delivery_area",
        # "electro",
        "facade",
        # "kicker",
        # "meadow",
        # "office",
        # "pipes",
        # "playground",
        # "relief",
        # "relief_2",
        # "terrace",
        # "terrains"
    ],
    'eth3d_raw_dslr_test': [
        "botanical_garden",
        "boulders",
        "bridge",
        "door",
        "exhibition_hall",
        "lecture_room",
        "living_room",
        "lounge",
        "observatory",
        "old_computer",
        "statue",
        "terrace_2",
    ],
    'blendedmvs': None,
    'blendedmvs+': None,
    'blendedmvs++': None,
}

times_colmap = {root: {} for root in datasets_full.keys()}
times_colmap["euroc"] = {
    "MH_01_easy" : 48914.0,
    "MH_02_easy" : 51894.24,
    "MH_03_medium" : 24453.3,
    "MH_04_difficult" : 13708.62,
    "MH_05_difficult" : 4525.68,
    "V1_01_easy" : 37055.76,
    "V1_02_medium" : 3269.04,
    "V1_03_difficult" : 2410.5,
    "V2_01_easy" : 9367.32,
    "V2_02_medium" : 3759.18,
    "V2_03_difficult" : 429.06,
}
times_colmap['lamar_map'] = {
    "CAB": 194033.6,
    "HGE": 249771.1,
    "LIN": 620176.38
}


reconstruction_file_names = ["cameras.bin", "images.bin", "points3D.bin"]

In [4]:
# Set up the base dir here
path_roots = {
    "euroc":
        "../../data/euroc/",
    "1dsfm_colmap":
        "../../data/1dsfm_colmap/",
    "KITTI":
        "../../data/KITTI/dataset/processed/",
    "eth3d_slam":
        "../../data/ETH3D/training/",
    "eth3d_dslr":
        "../../data/ETH3D/mvs/multi_view_training_dslr_undistorted/",
    "eth3d_rig":
        "../../data/ETH3D/mvs/multi_view_training_rig_undistorted/",
    "mip360":
        "../../data/mip360/",
    "lamar_map":
        "../../data/lamar_map/",
    "1dsfm_ori":
        "../../data/1dsfm_ori/",
    "1dsfm":
        "../../data/1dsfm/datasets/",
    "eth3d_dslr_test":
        "../../data/ETH3D/mvs/multi_view_test_dslr_undistorted/",
    "imc_haiper":
        "../../data/IMC2023/train/haiper/",
    "imc_heritage":
        "../../data/IMC2023/train/heritage/",
    "imc_urban":
        "../../data/IMC2023/train/urban/",
    "imc_phototourism":
        "../../data/IMC2023/train/phototourism/",
    "strecha":
        "../../data/strecha/",
    "imc2024":
        "../../data/IMC2024/train/",
    "imc2021_test":
        "../../data/IMC2021_test/phototourism/",
    "imc2021_val":
        "../../data/IMC2021_val/phototourism/",
    "eth3d_raw_dslr":
        "../../data/ETH3D/mvs/multi_view_training_dslr_jpg/",
    "eth3d_raw_dslr_test":
        "../../data/ETH3D/mvs/multi_view_test_dslr_jpg/",
    "blendedmvs":
        "../../../datasets/BlendedMVS/",
    "blendedmvs+":
        "../../../datasets/BlendedMVS+/",
    "blendedmvs++":
        "../../../datasets/BlendedMVS++/",
}

In [28]:
# TODO: change the configuration here
# euroc, mip360, eth3d_slam, eth3d_dslr, eth3d_dslr_test, eth3d_rig, imc_haiper, imc_heritage, imc_urban, imc_phototourism
# roots = ["mip360"]
# roots = ["lamar_map"]
# roots = ["eth3d_rig"]
# roots = ["strecha"]
# roots = ["eth3d_slam"]
# roots = ["eth3d_dslr", "eth3d_dslr_test"]
# roots = ["imc_haiper", "imc_heritage", "imc_urban", "imc_phototourism"]
# roots = ["imc_haiper", "imc_heritage", "imc_urban"]
# roots = ["imc_heritage"]
# roots = ["imc_haiper"]
# roots = ["imc_phototourism"]
# roots = ["imc2021_test"]
# roots = ["imc2024"]
# roots = ["eth3d_dslr"]
roots = ["eth3d_raw_dslr"]
# roots = ["eth3d_raw_dslr", "eth3d_raw_dslr_test"]
# roots = ["euroc"]
# roots = ["blendedmvs"]



# 3d, colmap
# post_fix = "bundled_iterative"
post_fix = "refactored"
# post_fix = "refactored_sparse"
# post_fix = "bundled_pipeline"
# post_fix = "test_gtrot"
# post_fix = "test_calibed_largermerge"


path_bases = [
    # "colmap_openmvg/reconstruction",
    # # # # "globalsfm_lud/aligned_cauchy", # for imc_haiper, imc_heritage, imc_urban, imc_phototourism, eth3d_dslr, eth3d_dslr_test
    # # # # # "globalsfm_lud/aligned", # for eth3d_slam, eth3d_rig, mip360
    # # "globalsfm_theia",
    # # # "globalsfm_theia_test",
    # # "globalsfm_hsfm",
    # # # "globalsfm_nonlinear/test_calibed",
    # # # "test/5",
    # # # "test_ra/5",
    # # # "globalsfm_nonlinear/test_calibed_noreest",
    # # # "globalsfm_nonlinear/test_calibed_oriF",
    # # # "globalsfm_nonlinear/pts_test/0",
    # # # "globalsfm_nonlinear/test",
    # # "globalsfm_nonlinear/bundled_pipeline",
    # # "globalsfm_nonlinear/bundled_iterative",
    # # # "globalsfm_nonlinear/test_gtrot",
    # # "globalsfm_nonlinear/" + post_fix + "_notriang",
    # # "globalsfm_theia",
    # "globalsfm_theia_refactored",
    # "globalsfm_hsfm_refactored",
    # "colmap_ligt/reconstruction",
    # # "globalsfm_hsfm",
    # # "globalsfm_nonlinear/" + post_fix,
    # # "globalsfm_nonlinear/refactored_lud",
    # # "globalsfm_nonlinear/refactored_onlycam",
    # # "globalsfm_nonlinear/refactored_ptcam",
    # "globalsfm_nonlinear/refactored_onlypt",
    "globalsfm_nonlinear/refactored_onlypt/0",
    # "globalsfm_nonlinear/nonadaptive_onlypt/0",
    # "globalsfm_nonlinear/adaptive_onlypt/0",
    # "globalsfm_nonlinear/rot_1dof/0",
    # "globalsfm_nonlinear/rot_3dof/0",
    # "globalsfm_nonlinear/ra_1dof/0",
    # "globalsfm_nonlinear/ra_3dof/0",
    # # "globalsfm_nonlinear/positioning_ptcam",
    # # "globalsfm_nonlinear/positioning_onlypt",
    # # "globalsfm_nonlinear/positioning_onlypt_new",
    # # "globalsfm_nonlinear/refactored_notriang",
    # # "globalsfm_nonlinear/refactored",
    # # "globalsfm_nonlinear/refactored_sparse_40_notriang",
    # # "globalsfm_nonlinear/refactored_sparse_40",
    # # "globalsfm_nonlinear/refactored_sparse_45_notriang",
    # # "globalsfm_nonlinear/refactored_sparse_45",
    # # "globalsfm_nonlinear/refactored_sparse_48_notriang",
    # # "globalsfm_nonlinear/bundled_pipeline",
    # # "globalsfm_nonlinear/bundled_nosparsify_1ra",
    # # "globalsfm_nonlinear/bundled_nosparsify",
    # # "globalsfm_nonlinear/pts/empty",
    "sparse/0",
    # "colmap",
    # # "hloc/sparse",
    # # "hloc/globalsfm_nonlinear/test_calibed_oriF",
    # # "sfm",
]

# path_bases = [
#     # "globalsfm_nonlinear/bundled_theia",
#     # "globalsfm_nonlinear/bundled_nopt",
#     # "globalsfm_nonlinear/bundled_ptcam",
#     # "globalsfm_nonlinear/" + post_fix,
#     "globalsfm_nonlinear/refactored_lud",
#     "globalsfm_nonlinear/refactored_onlycam",
#     "globalsfm_nonlinear/refactored_ptcam",
#     "globalsfm_nonlinear/refactored_onlypt",
# ]

# path_bases = ["globalsfm_nonlinear/test/0"]
# path_bases = [""]
# path_bases = ["sparse_resized/0", "glomap_resized/0", "mast3r_sfm/0"]
# path_bases = ["sparse/0", "globalsfm_nonlinear/refactored_onlypt", "e2esfm"]
# path_bases = ["glomap/0"]
# path_bases = ["globalsfm_nonlinear/glomap_stochastic_3it/0", "globalsfm_nonlinear/glomap_stochastic_0.5/0", "globalsfm_nonlinear/glomap_stochastic/0", "globalsfm_nonlinear/glomap_stochastic_0.1/0", "globalsfm_nonlinear/test/0"]
# path_bases = ["globalsfm_nonlinear/weight_random_gponly/0", "globalsfm_nonlinear/weight_uniform_gponly/0", "globalsfm_nonlinear/weight_trained_gponly/0", "globalsfm_nonlinear/weight_inliers_gponly/0"]

path_bases = ["colmap_thinprism/0", "colmap_radial/0", "implicit_sfm"]


In [29]:
# read in time

times_3d = {x: {} for x in datasets_full.keys()}
times_lud = {x: {} for x in datasets_full.keys()}
times_theia = {x: {} for x in datasets_full.keys()}
times_hsfm = {x: {} for x in datasets_full.keys()}
times_openmvg= {}
times_ligt= {}

target_string = "globalsfm_nonlinear"
post_fixes = [x.split("/")[1] if x is not None and x.find(target_string) >= 0 else None for x in path_bases]
if not post_fix in post_fixes:
    post_fixes.append(post_fix)

print(post_fixes)

for post_fix_temp in post_fixes:
    if post_fix_temp is None:
        continue
    times_3d[post_fix_temp] = {x: {} for x in datasets_full.keys()}
    timefile_names = ["../../build/time_trans.txt", "../../build/time_trans_postsubmission.txt", "../../../glomap_refactor/build/time_trans_refactored.txt"]
    for timefile_name in timefile_names:
        file = open(timefile_name, "r")
        while True:
            line = file.readline()
            if not line:
                break
            
            line = line.strip()
            if line == post_fix_temp:        
                line = file.readline()         
                while line.strip() != "":
                    line = line.strip()
                    times = line.split()
                    
                    if times[-2] == "1":
                        if len(times) == 3:
                            times_3d[post_fix_temp][times[-3]] = float(times[-1])
                        else:
                            times_3d[post_fix_temp][times[0]][times[-3]] = float(times[-1])
                    elif times[-2] == "2":
                        if len(times) == 3:
                            times_lud[times[-3]] = float(times[-1])
                        else:
                            times_lud[times[0]][times[-3]] = float(times[-1])

                    line = file.readline()
                    if not line:
                        break

file = open("time_colmap.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_colmap[line[0]][line[1]] = float(line[2])

file = open("time_colmap_mvs.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_colmap[line[0]][line[1]] = float(line[2])

file = open("time_colmap_mip360.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_colmap[line[0]][line[1]] = float(line[2])

file = open("time_colmap_imc.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_colmap[line[0]][line[1]] = float(line[2])

file = open("time_lud.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_lud[line[0]][line[1]] = float(line[-1])

# file = open("../../build/time_theia.txt", "r")
# data = file.readlines()
# data = [x.strip().split() for x in data]
# for line in data:
#     times_theia[line[0]][line[1]] = float(line[-1])

# file = open("../../build/time_hsfm.txt", "r")
# data = file.readlines()
# data = [x.strip().split() for x in data]
# for line in data:
#     times_hsfm[line[0]][line[1]] = float(line[-1])

file = open("../../../glomap_refactor/build/time_theia_refactored.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_theia[line[0]][line[1]] = float(line[-1])

file = open("../../../glomap_refactor/build/time_hsfm_refactored.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_hsfm[line[0]][line[1]] = float(line[-1])

if (len(path_bases) > 1 and path_bases[1].find("cauchy") >= 0):
    file = open("time_lud_colmap_cauchy.txt", "r")
else:
    file = open("time_lud_colmap.txt", "r")
    
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    if line[0] not in times_lud:
        times_lud[line[0]] = {}
    if line[1] not in times_lud[line[0]]:
        times_lud[line[0]][line[1]] = float(line[-1])
    else:
        times_lud[line[0]][line[1]] += float(line[-1])

file = open("../../../kapture/openmvg_time.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_openmvg[line[1]] = float(line[2])
    
file = open("../../../kapture/ligt_time.txt", "r")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    times_ligt[line[1]] = float(line[2])

# there is a annoying "s" at the end of the time, remove it
file = open("../../build/time_relpose.txt")
data = file.readlines()
data = [x.strip().split() for x in data]
for line in data:
    for post_fix_temp in post_fixes:
        if post_fix_temp is None:
            continue
        if post_fix_temp.find("refactored") >= 0:
            continue
        # if line[0] in times_3d[post_fix_temp] and line[1] in times_3d[post_fix_temp][line[0]]:
        #     times_3d[post_fix_temp][line[0]][line[1]] += float(line[-1][:-1])
    # if line[0] in times_theia and line[1] in times_theia[line[0]]:
    #     times_theia[line[0]][line[1]] += float(line[-1][:-1])
    # if line[0] in times_hsfm and line[1] in times_hsfm[line[0]]:
    #     times_hsfm[line[0]][line[1]] += float(line[-1][:-1])


print(post_fixes)
print(path_bases)
print(times_3d)
    

[None, None, None, 'refactored']
[None, None, None, 'refactored']
['colmap_thinprism/0', 'colmap_radial/0', 'implicit_sfm']
{'KITTI': {}, 'lamar': {}, '1dsfm_colmap': {}, 'euroc': {}, 'lamar_map': {}, 'eth3d_slam': {}, 'eth3d_dslr': {}, 'eth3d_rig': {}, 'mip360': {}, 'eth3d_dslr_test': {}, 'imc_haiper': {}, 'imc_heritage': {}, 'imc_urban': {}, 'imc_phototourism': {}, '1dsfm_ori': {}, 'flowmap': {}, 'strecha': {}, 'imc2024': {}, 'imc2021_test': {}, 'imc2021_val': {}, 'eth3d_raw_dslr': {}, 'eth3d_raw_dslr_test': {}, 'blendedmvs': {}, 'blendedmvs+': {}, 'blendedmvs++': {}, 'refactored': {'KITTI': {}, 'lamar': {}, '1dsfm_colmap': {}, 'euroc': {}, 'lamar_map': {'CAB': 6162.24, 'HGE': 12587.2, 'LIN': 18466.6}, 'eth3d_slam': {'cables_1': 499.254, 'cables_2': 12.1554, 'cables_3': 26.3364, 'camera_shake_1': 7.5131, 'camera_shake_2': 11.6773, 'camera_shake_3': 2.7526, 'ceiling_1': 39.4557, 'ceiling_2': 164.105, 'desk_3': 145.981, 'desk_changing_1': 173.527, 'einstein_1': 147.958, 'einstein_2': 5

In [4]:
# results for the relative pose
results = {}
results_r = {}
total_image = {}
errors_r_all = []
errors_t_all = []
for root in roots:
    results[root] = {}
    results_r[root] = {}
    total_image[root] = {}
    if datasets_full[root] is None:
        datasets_full[root] = sorted([x for x in os.listdir(path_roots[root]) if os.path.isdir(path_roots[root] + x)], key=locale.strxfrm)
    for dataset in datasets_full[root]:
        dir_base = path_roots[root] + dataset + "/"

        if root == "strecha":
            dir_base = path_roots[root] + dataset + "/colmap/"
        if root == "imc2021_test":
            dir_base = path_roots[root] + dataset + "/set_100/"

        print(dataset)

        results[root][dataset] = []
        results_r[root][dataset] = []
        database_path = os.path.join(dir_base, "database.db")
        image_path = os.path.join(dir_base, "images")

        for path_base in path_bases:
            if path_base is None:
                results[root][dataset].append(None)
                results_r[root][dataset].append(None)
                continue
            
            if not os.path.exists(os.path.join(dir_base, path_base, "cameras.txt")) and not os.path.exists(os.path.join(dir_base, path_base, "cameras.bin")):
                print(os.path.join(dir_base, path_base), "not possible")
                results[root][dataset].append(None)
                results_r[root][dataset].append(None)
                continue
            
            # Only use the txt files as they are directly generated
            for name in reconstruction_file_names:
                if os.path.exists(os.path.join(dir_base, path_base, name)) and os.path.exists(os.path.join(dir_base, path_base, name[:-4]+ ".txt")):
                    os.remove(os.path.join(dir_base, path_base, name))
                    print("removed", os.path.join(dir_base, path_base, name))
                    
            status = True
            key_words = ["sparse", "colmap_thinprism", "colmap_radial"]
            for key_word in key_words:
                if path_base.find(key_word + "/0") >= 0:
                    total_dirs = sorted([x for x in os.listdir(dir_base + key_word + "") if os.path.isdir(dir_base + key_word + "/" + x)])
                    if len(total_dirs) > 1:
                        # num_imgs = []
                        max_num_img = 0
                        max_idx = -1
                        for idx, sub_dir in enumerate(total_dirs):
                            reconstruction = pycolmap.Reconstruction()
                            reconstruction.read(dir_base + key_word + "/" + sub_dir)
                            if reconstruction.num_images() > max_num_img:
                                max_num_img = reconstruction.num_images()
                                max_idx = idx
                        if max_idx != 0:
                            status = False
                            path_base = key_word + "/" + str(max_idx)
                            print("path_base:", path_base)
            
            if path_base.find("colmap_openmvg") >= 0 or path_base.find("colmap_ligt") >= 0:
                reconstruction = pycolmap.Reconstruction()
                reconstruction.read(dir_base + path_base)
                for id in reconstruction.reg_image_ids():
                    image_name = reconstruction.images[id].name
                    if image_name.find("records_data/") >= 0:
                        reconstruction.images[id].name = image_name.replace("records_data/", "")
                reconstruction.write_text(dir_base + path_base)

            file = open(dir_base + "centers_gt.txt", "r")
            centers_gt = {}
            names = []
            centers = []
            for line in file:
                line = line.split()
                centers_gt[line[0]] = np.array([float(line[1]), float(line[2]), float(line[3])])
                names.append(line[0])
                centers.append(centers_gt[line[0]])


            file = open(dir_base + "gtpose_trans.txt", "r")
            poses_gt = {}
            for line in file:
                line = line.split()
                poses_gt[line[0]] = poselib.CameraPose()
                poses_gt[line[0]].q = np.array([float(line[1]), float(line[2]), float(line[3]), float(line[4])])
                poses_gt[line[0]].t = np.array([float(line[5]), float(line[6]), float(line[7])])

            reconstruction = pycolmap.Reconstruction()
            reconstruction.read(dir_base + path_base)

            poses_calc = {}
            for image_name in centers_gt.keys():
                image = reconstruction.find_image_with_name(image_name)
                if image is not None:
                    pose = poselib.CameraPose()
                    pose.q = image.cam_from_world.rotation.quat[[3,0,1,2]]
                    pose.t = image.cam_from_world.translation
                    poses_calc[image_name] = pose

            poses_gt_list = np.array([poses_gt[x] for x in sorted(poses_gt.keys())])
            poses_calc_list = np.array([poses_calc[x] if x in poses_calc else None for x in sorted(poses_gt.keys())])
                
            # centers_est = {}
            errors_r = []
            errors_t = []
            pose = poselib.CameraPose()
            image_names = sorted(list(centers_gt.keys()))

            
            errors_r_ori = np.array(errors_r)
            errors_t_ori = np.array(errors_t)

            errors_r, errors_t = calc_pairwise_relative_error_batch(poses_gt_list, poses_calc_list)

            errors_r_all += list(errors_r)
            errors_t_all += list(errors_t)
            
            errors_t = np.array(errors_t)
            # errors_r = np.array(errors_r)
            results[root][dataset].append(np.maximum(errors_r, errors_t))
            # results[root][dataset].append(errors_r)
            results_r[root][dataset].append(errors_r)
            total_image[root][dataset] = len(centers_gt)
            


NameError: name 'roots' is not defined

In [27]:
# printing method, color-coded

roots_rel = ["eth3d_rig", "mip360", "eth3d_dslr", "eth3d_dslr_test", "imc_haiper", "imc_heritage", "imc_urban", "imc_phototourism", "imc2024", "KITTI", "euroc", "eth3d_raw_dslr", "blendedmvs"]
roots_abs = ["lamar_map", "eth3d_slam", "strecha"]

results_full = [[] for i in range(len(path_bases))]
results_table = []
for root in roots:
    if root in roots_abs:
        mode = "abs"
    elif root in roots_rel:
        mode = "rel"
    else:
        print("NOT KNOWN!!!")
    
    # mode = 'abs'
    if mode == "rel":
        if root.find("eth3d_dslr") >= 0:
            threshold = 1
            AUC_thres =  [1, 3, 5]
            # threshold = 0.02
            # AUC_thres =  [0.02, 0.05, 0.1]
        elif root == "eth3d_rig":
            threshold = 1
            AUC_thres = [1, 3, 5]
            # threshold = 0.02
            # AUC_thres =  [0.02, 0.05, 0.1]
        elif root == "eth3d_slam":
            # threshold = 1
            # AUC_thres = [3, 5, 10]
            threshold = 0.02
            AUC_thres =  [0.02, 0.05, 0.1]
        elif root.find("imc") >= 0:
            threshold = 3
            AUC_thres = [3, 5, 10]
        elif root == "mip360":
            threshold = 3
            AUC_thres = [3, 5, 10]
        elif root == "KITTI":
            threshold = 1
            AUC_thres = [0.5, 1, 2]
        elif root == "euroc":
            threshold = 1
            AUC_thres = [0.5, 1, 2]
        else:
            threshold = 1
            AUC_thres = [1, 3, 5]
        idx = list(range(len(AUC_thres) + 2))[1:]
    elif mode == "abs":
        if root == "lamar_map":
            threshold = 1
            AUC_thres = [1, 5]
        elif root == "eth3d_slam":
            threshold = 0.1
            AUC_thres = [0.1, 0.5]
        else:
            threshold = 0.1
            AUC_thres = [0.1, 0.5]
        idx = list(range(len(AUC_thres) + 2))
    print(threshold, AUC_thres)

    results_class = {}
    for dataset in datasets_full[root]:
        if root == "eth3d_slam":
            prefix = dataset.split("_")[0]
        else:
            prefix = dataset
        # prefix = dataset

        if prefix not in results_class:
            results_class[prefix] = {path_base: [] for path_base in path_bases}

        result_row = []
        times_row = []
        for i, path_base in enumerate(path_bases):
            if path_base is None:
                continue
            if results[root][dataset][i] is None:
                continue
            inliers = results[root][dataset][i] < threshold

            row = []
            row.append(np.average(results[root][dataset][i] < threshold) * 100)
            row += compute_auc(results[root][dataset][i], AUC_thres)
            result_row.append(row)

            if path_base == "colmap" or path_base == "colmap/0" or path_base.find("sparse") >= 0:
                if dataset in times_colmap[root]:
                    result_row[-1].append(times_colmap[root][dataset])
                else:
                    result_row[-1].append(1e10)
                # result_row[-1].append(times_colmap[root][dataset])
            elif path_base.find("theia") != -1:
                if dataset in times_theia:
                    result_row[-1].append(times_theia[dataset])
                elif dataset in times_theia[root]:
                    result_row[-1].append(times_theia[root][dataset])
                else:
                    result_row[-1].append(1e10)
            elif path_base.find("openmvg") != -1:
                result_row[-1].append(times_openmvg[dataset])
            elif path_base.find("ligt") != -1:
                result_row[-1].append(times_ligt[dataset])
            elif post_fixes[i] is not None:
                if post_fixes[i] in times_3d:
                    if dataset in times_3d[post_fixes[i]][root]:
                        result_row[-1].append(times_3d[post_fixes[i]][root][dataset])
                    elif dataset in times_3d[post_fixes[i]]:
                        result_row[-1].append(times_3d[post_fixes[i]][dataset])
                    else:
                        result_row[-1].append(1e10)
                else:
                    result_row[-1].append(1e10)
            else:
                if post_fix in times_3d:
                    if dataset in times_3d[post_fix][root]:
                        result_row[-1].append(times_3d[post_fix][root][dataset])
                    else:
                        result_row[-1].append(1e10)
                else:
                    result_row[-1].append(1e10)
            
            results_class[prefix][path_base].append(result_row[-1])
            results_full[i].append(result_row[-1])

        # result_row = np.asarray(result_row)
        # result_row = np.round(result_row, 2)
    # print(results_class)
    
    prefix_full = sorted(list(results_class.keys()))
    for prefix in prefix_full:
        print(prefix.replace("_", "\\_"), end=" ")
        result_row = []
        max_rows = np.max([len(results_class[prefix][path_base]) for path_base in path_bases])
        for path_base in path_bases:
            if len(results_class[prefix][path_base]) == 0:
                result_row.append(np.zeros((len(AUC_thres) + 2,)))
                result_row[-1][-1] = 1e10
                continue
            result_row.append(np.sum(np.asarray(results_class[prefix][path_base]), axis=0) / max_rows)
            result_row[-1][-1] = np.mean(np.asarray(results_class[prefix][path_base])[:,-1], axis=0)
        

        result_row = np.asarray(result_row)
        result_row = np.round(result_row, 1)

        ranks = [rankify(result_row[:, i], 1) for i in range(result_row.shape[1])]
        ranks[-1] = rankify(result_row[:, -1], -1)

        
        for j in idx[:-1]:
            print("&", end="")
            counter = 0
            for i, path_base in enumerate(path_bases):
                if prefix in results[root] and results[root][prefix][i] is None:
                    print("& -", end=" ")
                    counter += 1
                    continue
                    
                if ranks[j][counter] == 0:
                    print("& \\cellcolor{tabfirst}" + "{:.1f}".format(result_row[i][j]), end=" ")
                elif ranks[j][counter] == 1:
                    print("& \\cellcolor{tabsecond}" + "{:.1f}".format(result_row[i][j]), end=" ")
                # elif ranks[j][counter] == 2:
                #     print("& \\cellcolor{tabthird}" + "{:.1f}".format(result_row[i][j]), end=" ")

                # if ranks[j][counter] == 0:
                #     print("& \\textbf{" + "{:.1f}".format(result_row[i][j]) + "}", end=" ")
                # elif ranks[j][counter] == 1:
                #     print("& \\textit{" + "{:.1f}".format(result_row[i][j]) + "}", end=" ")
                # elif ranks[j][counter] == 2:
                #     print("& \\cellcolor{tabthird}" + "{:.1f}".format(result_row[i][j]), end=" ")
                else:
                    print("& " + "{:.1f}".format(result_row[i][j]), end=" ")
                counter += 1
        print("\\\\")
                
        # for j in idx:
        #     counter = 0
        #     for i, path_base in enumerate(path_bases):
        #         if prefix in results[root] and results[root][prefix][i] is None:
        #             print("& -", end=" ")
        #             counter += 1
        #             continue
                    
        #         print(" " + "{:.1f}".format(result_row[i][j]), end=" ")
        #         counter += 1
            
        # print()
        

result_full_average = []
result_full_average_valid = []
results_test = []

total_dataset = 0
for root in roots:
    total_dataset += len(datasets_full[root])
for i in range(len(path_bases)):
    if len(results_full[i]) == 0:
        result_full_average.append(np.nan * np.zeros((len(idx),)))
        continue

    results_test.append(np.asarray(results_full[i])[0])

    result_full_np = np.asarray(results_full[i])
    result_full_average.append(np.sum(result_full_np, axis=0) / total_dataset)
    result_full_average[-1][-1] = np.mean(result_full_np[:, -1])
    result_full_average_valid.append(np.mean(result_full_np, axis=0))

result_full_average = np.asarray(result_full_average)
result_full_average_valid = np.asarray(result_full_average_valid)

result_full_average = np.round(result_full_average, 1)
result_full_average_valid = np.round(result_full_average_valid, 1)
ranks_ave = [rankify(result_full_average[:, i], 1) for i in range(result_full_average.shape[1])]
ranks_ave[-1] = rankify(result_full_average[:, -1], -1)

print("\\midrule")
print("\\textit{Average}", end=" ")

for j in idx:
    print("&", end="")
    counter = 0
    for i, path_base in enumerate(path_bases):
        if np.isnan(result_full_average[i][0]):
            print("& " + "-", end=" ")
            counter += 1
            continue
        if ranks_ave[j][counter] == 0:
            print("& \\cellcolor{tabfirst}" + "{:.1f}".format(result_full_average[i][j]), end=" ")
        elif ranks_ave[j][counter] == 1:
            print("& \\cellcolor{tabsecond}" + "{:.1f}".format(result_full_average[i][j]), end=" ")
        # elif ranks_ave[j][counter] == 2:
        #     print("& \\cellcolor{tabthird}" + "{:.1f}".format(result_full_average[i][j]), end=" ")
        # if ranks_ave[j][counter] == 0:
        #     print("& \\textbf{" + "{:.1f}".format(result_full_average[i][j]) + "}", end=" ")
        # elif ranks_ave[j][counter] == 1:
        #     print("& \\t
        # extit{" + "{:.1f}".format(result_full_average[i][j]) + "}", end=" ")
        else:
            print("& " + "{:.1f}".format(result_full_average[i][j]), end=" ")
        # print("& " + "{:.1f}".format(result_full_average[i][j]), end=" ")delivery\_area && \cellcolor{tabsecond}92.3 & \cellcolor{tabfirst}92.8 & 87.6 && \cellcolor{tabsecond}97.4 & \cellcolor{tabfirst}97.6 & 95.9 && \cellcolor{tabsecond}98.5 & \cellcolor{tabfirst}98.6 & 97.5 \\

            
        counter += 1
print("\\\\")


1 [1, 3, 5]
courtyard && \cellcolor{tabfirst}55.9 & \cellcolor{tabsecond}25.5 && \cellcolor{tabfirst}83.0 & \cellcolor{tabsecond}62.4 && \cellcolor{tabfirst}89.8 & \cellcolor{tabsecond}75.2 \\
facade && \cellcolor{tabfirst}72.8 & \cellcolor{tabsecond}63.3 && \cellcolor{tabfirst}88.3 & \cellcolor{tabsecond}84.2 && \cellcolor{tabfirst}91.9 & \cellcolor{tabsecond}89.3 \\
\midrule
\textit{Average} && \cellcolor{tabfirst}64.4 & \cellcolor{tabsecond}44.4 && \cellcolor{tabfirst}85.7 & \cellcolor{tabsecond}73.3 && \cellcolor{tabfirst}90.8 & \cellcolor{tabsecond}82.2 && \cellcolor{tabfirst}10000000000.0 & \cellcolor{tabfirst}10000000000.0 \\


## Results for SLAM

In [16]:
realign = True

results = {}
results_r = {}
total_image = {}
errors_r_all = []
errors_t_all = []

for root in roots:
    results[root] = {}
    results_r[root] = {}
    total_image[root] = {}
    for dataset in datasets_full[root]:
        dir_base = path_roots[root] + dataset + "/"
        if root == "strecha":
            dir_base = path_roots[root] + dataset + "/colmap/"
        
        print(dataset)

        results[root][dataset] = []
        results_r[root][dataset] = []
        database_path = os.path.join(dir_base, "database.db")
        image_path = os.path.join(dir_base, "images")

        for path_base in path_bases:
            if path_base is None:
                results[root][dataset].append(None)
                results_r[root][dataset].append(None)
                continue
            
            if not os.path.exists(os.path.join(dir_base, path_base, "cameras.txt")) and not os.path.exists(os.path.join(dir_base, path_base, "cameras.bin")):
                print(os.path.join(dir_base, path_base), "not possible")
                results[root][dataset].append(None)
                results_r[root][dataset].append(None)
                continue
            
            status = True
            if path_base.find("sparse/0") >= 0:
                total_dirs = sorted([x for x in os.listdir(dir_base + "sparse") if os.path.isdir(dir_base + "sparse/" + x)])
                if len(total_dirs) > 1:
                    # num_imgs = []
                    max_num_img = 0
                    max_idx = -1
                    for idx, sub_dir in enumerate(total_dirs):
                        reconstruction = pycolmap.Reconstruction()
                        reconstruction.read(dir_base + "sparse/" + sub_dir)
                        if reconstruction.num_images() > max_num_img:
                            max_num_img = reconstruction.num_images()
                            max_idx = idx
                    if max_idx != 0:
                        status = False
                        path_base = "sparse/" + str(max_idx)
                        print("path_base:", path_base)
            
            file = open(dir_base + "centers_gt.txt", "r")
            centers_gt = {}
            names = []
            centers = []
            for line in file:
                line = line.split()
                centers_gt[line[0]] = np.array([float(line[1]), float(line[2]), float(line[3])])
                names.append(line[0])
                centers.append(centers_gt[line[0]])

            if realign:
                for name in reconstruction_file_names:
                    if os.path.exists(os.path.join(dir_base, path_base, name)) and os.path.exists(os.path.join(dir_base, path_base, name[:-4]+ ".txt")):
                        os.remove(os.path.join(dir_base, path_base, name))
                        print("removed", os.path.join(dir_base, path_base, name))

                if path_base.find("colmap_openmvg") >= 0 or path_base.find("colmap_ligt") >= 0:
                    reconstruction = pycolmap.Reconstruction()
                    reconstruction.read_text(dir_base + path_base)
                    for id in reconstruction.reg_image_ids():
                        image_name = reconstruction.images[id].name
                        if image_name.find("records_data/") >= 0:
                            reconstruction.images[id].name = image_name.replace("records_data/", "")
                    reconstruction.write_text(dir_base + path_base)
                

                opt = pycolmap.RANSACOptions()
                if root == "1dsfm_colmap":
                    opt.max_error = 0.5
                elif root == "KITTI":
                    opt.max_error = 5.0
                elif root == "1dsfm":
                    opt.max_error = 10.
                elif root == "lamar_map":
                    opt.max_error = 1.
                else:
                    opt.max_error = 0.1
                try:
                    reconstruction = pycolmap.Reconstruction(dir_base + path_base)
                    trans = pycolmap.align_reconstrution_to_locations(reconstruction, names, centers, 3, opt)
                    reconstruction.transform(trans)
                    reconstruction.write_text(dir_base + path_base)
                except:
                    print("Error in aligning")
                    results[root][dataset].append(None)
                    results_r[root][dataset].append(None)
                    continue

            # for i in range(len(centers)):
            #     print(centers_gt[names[i]], trans centers[i])

            file = open(dir_base + "gtpose_trans.txt", "r")
            poses_gt = {}
            for line in file:
                line = line.split()
                poses_gt[line[0]] = poselib.CameraPose()
                poses_gt[line[0]].q = np.array([float(line[1]), float(line[2]), float(line[3]), float(line[4])])
                poses_gt[line[0]].t = np.array([float(line[5]), float(line[6]), float(line[7])])

                
            reconstruction = pycolmap.Reconstruction()
            reconstruction.read_text(dir_base + path_base)
            # centers_est = {}
            errors_r = []
            errors_t = []
            pose = poselib.CameraPose()
            for image_name in centers_gt.keys():
                image = reconstruction.find_image_with_name(image_name)
                if image is not None:
                    center = image.projection_center()
                    errors_t.append(np.linalg.norm(center - centers_gt[image_name]))
                    pose.q = image.cam_from_world.rotation.quat[[3,0,1,2]]
                    pose.t = image.cam_from_world.translation
                    # errors_t.append(np.linalg.norm(pose.t - poses_gt[image_name].t))
                    errors_r.append(calc_angle(pose, poses_gt[image_name]))
                else:
                    errors_t.append(100)
                    errors_r.append(180)
            

            errors_r_all += errors_r
            errors_t_all += errors_t
            errors_t = np.array(errors_t)
            errors_r = np.array(errors_r)
            results[root][dataset].append(errors_t)
            results_r[root][dataset].append(errors_r)
            total_image[root][dataset] = len(centers_gt)


LIN


In [44]:
# printing method, color-coded (for transposed)

roots_rel = ["eth3d_rig", "mip360", "eth3d_dslr", "eth3d_dslr_test", "imc_haiper", "imc_heritage", "imc_urban", "imc_phototourism", "imc2024"]
roots_abs = ["lamar_map", "eth3d_slam", "strecha"]

results_full = [[] for i in range(len(path_bases))]
results_table = []
for root in roots:
    if root in roots_abs:
        mode = "abs"
    elif root in roots_rel:
        mode = "rel"
    else:
        print("NOT KNOWN!!!")
    
    # mode = 'abs'
    if mode == "rel":
        if root.find("eth3d_dslr") >= 0:
            threshold = 1
            AUC_thres =  [1, 3, 5]
            # threshold = 0.02
            # AUC_thres =  [0.02, 0.05, 0.1]
        elif root == "eth3d_rig":
            threshold = 1
            AUC_thres = [1, 3, 5]
            # threshold = 0.02
            # AUC_thres =  [0.02, 0.05, 0.1]
        elif root == "eth3d_slam":
            # threshold = 1
            # AUC_thres = [3, 5, 10]
            threshold = 0.02
            AUC_thres =  [0.02, 0.05, 0.1]
        elif root.find("imc") >= 0:
            threshold = 3
            AUC_thres = [3, 5, 10]
        elif root == "mip360":
            threshold = 3
            AUC_thres = [3, 5, 10]
        else:
            threshold = 1
            AUC_thres = [1, 3, 5]
        idx = list(range(len(AUC_thres) + 2))[1:]
    elif mode == "abs":
        if root == "lamar_map":
            threshold = 1
            AUC_thres = [1, 5]
        elif root == "eth3d_slam":
            threshold = 0.1
            AUC_thres = [0.1, 0.5]
        else:
            threshold = 0.1
            AUC_thres = [0.1, 0.5]
        idx = list(range(len(AUC_thres) + 2))
    print(threshold, AUC_thres)

    results_class = {}
    for dataset in datasets_full[root]:
        if root == "eth3d_slam":
            prefix = dataset.split("_")[0]
        else:
            prefix = dataset
        # prefix = dataset

        if prefix not in results_class:
            results_class[prefix] = {path_base: [] for path_base in path_bases}

        result_row = []
        times_row = []
        for i, path_base in enumerate(path_bases):
            if path_base is None:
                continue
            if results[root][dataset][i] is None:
                continue
            inliers = results[root][dataset][i] < threshold

            row = []
            row.append(np.average(results[root][dataset][i] < threshold) * 100)
            row += compute_auc(results[root][dataset][i], AUC_thres)
            result_row.append(row)

            if path_base == "colmap" or path_base == "colmap/0" or path_base.find("sparse") >= 0:
                if dataset in times_colmap[root]:
                    result_row[-1].append(times_colmap[root][dataset])
                else:
                    result_row[-1].append(1e10)
                # result_row[-1].append(times_colmap[root][dataset])
            elif path_base.find("theia") != -1:
                if dataset in times_theia:
                    result_row[-1].append(times_theia[dataset])
                elif dataset in times_theia[root]:
                    result_row[-1].append(times_theia[root][dataset])
                else:
                    result_row[-1].append(1e10)
            elif path_base.find("hsfm") != -1:
                if dataset in times_hsfm:
                    result_row[-1].append(times_hsfm[dataset])
                elif dataset in times_hsfm[root]:
                    result_row[-1].append(times_hsfm[root][dataset])
                else:
                    result_row[-1].append(1e10)
            elif path_base.find("openmvg") != -1:
                result_row[-1].append(times_openmvg[dataset])
            elif path_base.find("ligt") != -1:
                result_row[-1].append(times_ligt[dataset])
            elif post_fixes[i] is not None:
                if post_fixes[i] in times_3d:
                    if dataset in times_3d[post_fixes[i]][root]:
                        result_row[-1].append(times_3d[post_fixes[i]][root][dataset])
                    elif dataset in times_3d[post_fixes[i]]:
                        result_row[-1].append(times_3d[post_fixes[i]][dataset])
                    else:
                        result_row[-1].append(1e10)
                else:
                    result_row[-1].append(1e10)
            else:
                if post_fix in times_3d:
                    if dataset in times_3d[post_fix][root]:
                        result_row[-1].append(times_3d[post_fix][root][dataset])
                    else:
                        result_row[-1].append(1e10)
                else:
                    result_row[-1].append(1e10)
            
            results_class[prefix][path_base].append(result_row[-1])
            results_full[i].append(result_row[-1])

        # result_row = np.asarray(result_row)
        # result_row = np.round(result_row, 2)
    # print(results_class)
    
    prefix_full = sorted(list(results_class.keys()))
    for prefix in prefix_full:
        # print(prefix.replace("_", "\\_"), end=" ")
        result_row = []
        max_rows = np.max([len(results_class[prefix][path_base]) for path_base in path_bases])
        for path_base in path_bases:
            if len(results_class[prefix][path_base]) == 0:
                result_row.append(np.zeros((len(AUC_thres) + 2,)))
                result_row[-1][-1] = 1e10
                continue
            result_row.append(np.sum(np.asarray(results_class[prefix][path_base]), axis=0) / max_rows)
            result_row[-1][-1] = np.mean(np.asarray(results_class[prefix][path_base])[:,-1], axis=0)
        

        result_row = np.asarray(result_row)
        result_row = np.round(result_row, 1)

        ranks = [rankify(result_row[:, i], 1) for i in range(result_row.shape[1])]
        ranks[-1] = rankify(result_row[:, -1], -1)

        
        # for i, path_base in enumerate(path_bases):
        #     print("&", end="")
        #     counter = 0
        #     for j in idx:
        #         if prefix in results[root] and results[root][prefix][i] is None:
        #             print("& -", end=" ")
        #             counter += 1
        #             continue
                    
        #         if ranks[j][counter] == 0:
        #             print("& \\textbf{" + "{:.1f}".format(result_row[i][j]) + "}", end=" ")
        #         else:
        #             print("& " + "{:.1f}".format(result_row[i][j]), end=" ")
        #         counter += 1
            
        # print("\\\\")
        

result_full_average = []
result_full_average_valid = []
results_test = []

total_dataset = 0
for root in roots:
    print(root)
    total_dataset += len(datasets_full[root])
for i in range(len(path_bases)):
    if len(results_full[i]) == 0:
        result_full_average.append(np.nan * np.zeros((len(idx),)))
        continue

    results_test.append(np.asarray(results_full[i])[0])

    result_full_np = np.asarray(results_full[i])
    result_full_average.append(np.sum(result_full_np, axis=0) / total_dataset)
    result_full_average[-1][-1] = np.mean(result_full_np[:, -1])
    result_full_average_valid.append(np.mean(result_full_np, axis=0))

result_full_average = np.asarray(result_full_average)
result_full_average_valid = np.asarray(result_full_average_valid)

result_full_average = np.round(result_full_average, 1)
result_full_average_valid = np.round(result_full_average_valid, 1)
ranks_ave = [rankify(result_full_average[:, i], 1) for i in range(result_full_average.shape[1])]
ranks_ave[-1] = rankify(result_full_average[:, -1], -1)


for i, path_base in enumerate(path_bases):
    print(path_base, end=" ")
    print("&", end="")
    for j in idx:
        if np.isnan(result_full_average[i][j]):
            print("& " + "-", end=" ")
            continue
        # if ranks_ave[j][i] == 0:
        #     print("& \\textbf{" + "{:.1f}".format(result_full_average[i][j]) + "}", end=" ")

        # if ranks_ave[j][i] == 0:
        #     print("& \\cellcolor{tabfirst}" + "{:.1f}".format(result_full_average[i][j]), end=" ")
        # elif ranks_ave[j][i] == 1:
        #     print("& \\cellcolor{tabsecond}" + "{:.1f}".format(result_full_average[i][j]), end=" ")
        # else:
        #     print("& " + "{:.1f}".format(result_full_average[i][j]), end=" ")
        print("& " + "{:.1f}".format(result_full_average[i][j]), end=" ")
    print("\\\\")
            


1 [1, 3, 5]
&& \textbf{33.9} 

IndexError: index 1 is out of bounds for axis 0 with size 1

In [356]:
# data = np.loadtxt("../../../instant-ngp/scripts/results_arxiv.txt")
data = np.loadtxt("../../../instant-ngp/scripts/results_final.txt")

datasets = datasets_full['mip360']

# openmvg, glomap, theia, colmap
# methods = ["theia"]
methods = ["openmvg", "theia", "glomap", "colmap"]


results_psnr = []
results_ssim = []
# = data[:]

# for i, dataset in enumerate(datasets):
for i, method in enumerate(methods):
    results_psnr.append(data[i::len(methods), 0])
    results_ssim.append(data[i::len(methods), 1])

results_psnr = np.array(results_psnr).T
results_ssim = np.array(results_ssim).T

results = [results_psnr, results_ssim]
results[0] = np.concatenate([results[0], np.mean(results_psnr, axis=0, keepdims=True)], axis=0)
results[1] = np.concatenate([results[1], np.mean(results_ssim, axis=0, keepdims=True)], axis=0)

precisions = [2, 3]

# bold a number if is the largest in the row
for i, dataset in enumerate(datasets + ["Average"]):
    print(dataset, end=" ")
    results[0][i] = np.round(results[0][i], precisions[0])
    results[1][i] = np.round(results[1][i], precisions[1])
    ranks = [rankify(results[0][i], 1), rankify(results[1][i], 1)]
    for k in range(2):
        print("&", end="")
        for j, method in enumerate(methods):
            # print(results[0])
            print("& ", end="")
            if ranks[k][j] == 0:
                print("\\cellcolor{tabfirst}", end="")
            elif ranks[k][j] == 1:
                print("\\cellcolor{tabsecond}", end="")
            print(("{:" + f'.{precisions[k]}f' + "}").format(results[k][i, j]), end=" ")
    
    print("\\\\")
    if i == len(datasets) - 1:
        print("\\midrule")


bicycle && 23.01 & 17.75 & \cellcolor{tabsecond}23.13 & \cellcolor{tabfirst}23.15 && 0.526 & 0.352 & \cellcolor{tabsecond}0.531 & \cellcolor{tabfirst}0.532 \\
bonsai && 23.88 & 28.54 & \cellcolor{tabfirst}30.36 & \cellcolor{tabsecond}29.66 && 0.767 & 0.872 & \cellcolor{tabfirst}0.904 & \cellcolor{tabsecond}0.896 \\
counter && 26.76 & \cellcolor{tabsecond}26.78 & 26.72 & \cellcolor{tabfirst}26.81 && 0.835 & \cellcolor{tabsecond}0.836 & 0.835 & \cellcolor{tabfirst}0.837 \\
garden && \cellcolor{tabsecond}24.97 & 20.19 & \cellcolor{tabsecond}24.97 & \cellcolor{tabfirst}24.98 && \cellcolor{tabsecond}0.653 & 0.456 & \cellcolor{tabfirst}0.655 & \cellcolor{tabsecond}0.653 \\
kitchen && \cellcolor{tabsecond}29.32 & 29.02 & \cellcolor{tabfirst}29.35 & 29.23 && \cellcolor{tabsecond}0.853 & 0.841 & \cellcolor{tabfirst}0.855 & 0.851 \\
room && 19.11 & 17.07 & \cellcolor{tabfirst}29.41 & \cellcolor{tabsecond}29.14 && 0.691 & 0.643 & \cellcolor{tabfirst}0.876 & \cellcolor{tabsecond}0.871 \\
stump && 

In [22]:

for dataset in ["Herzjesu", "Herzjesu_large", "Fountain", "Entry", "Castle", "Castle_large"]:
    if root == "eth3d_slam":
        prefix = dataset.split("_")[0]
    else:
        prefix = dataset
    # prefix = dataset

    if prefix not in results_class:
        results_class[prefix] = {path_base: [] for path_base in path_bases}

    result_row = []
    times_row = []
    for i, path_base in enumerate(path_bases):
        if path_base is None:
            continue
        if results[root][dataset][i] is None:
            continue
        inliers = results[root][dataset][i] < threshold

        print(dataset, np.round(np.average(results[root][dataset][i] * 1000), 2))

        # row = []
        # row.append(np.average(results[root][dataset][i] < threshold) * 100)
        # row += compute_auc(results[root][dataset][i], AUC_thres)
        # result_row.append(row)

        
        # results_class[prefix][path_base].append(result_row[-1])
        # results_full[i].append(result_row[-1])

    # result_row = np.asarray(result_row)
    # result_row = np.round(result_row, 2)
# print(results_class)

Herzjesu 4.13
Herzjesu_large 5.4
Fountain 2.79
Entry 6.32
Castle 24.95
Castle_large 22.36
